In [ ]:
! pip install -U openai-whisper

In [59]:
# cd drive/MyDrive/sonny

[Errno 2] No such file or directory: 'drive/MyDrive/sonny'
/content/drive/MyDrive/sonny


In [2]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio

from tqdm.notebook import tqdm


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [16]:
from torch.utils.data import Dataset
from torch.nn import functional as F
import random
import torch
import numpy as np
import pandas as pd
import soundfile as sf

In [104]:
class customAudioDataset(Dataset):
      def __init__(self, csv_path):
            self.data = pd.read_csv(csv_path, header=[0, 1])
            self.text_data = self.data['text_data'][' '].values
            self.wav_dir = self.data['wav_dir'][' '].values

      def __len__(self):
            return len(self.data)
          
      def __getitem__(self, idx):
            audio_input, sample_rate = sf.read(self.wav_dir[idx])
            audio = whisper.pad_or_trim(audio_input)
            audio = torch.tensor(audio, dtype = torch.float32)
            mel = whisper.log_mel_spectrogram(audio)
            return (mel, self.text_data[idx])

In [105]:
dataset = customAudioDataset('KEMDy20_val_data.csv')

In [107]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16)

In [ ]:
model = whisper.load_model("base")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

In [71]:
options = whisper.DecodingOptions(language="ko", without_timestamps=True)

In [110]:
hypotheses = []
references = []

for mels, texts in tqdm(loader):
    mels = mels.to('cuda')
    results = model.decode(mels, options)
    hypotheses.extend([result.text for result in results])
    references.extend(texts)

  0%|          | 0/125 [00:00<?, ?it/s]

In [111]:
data = pd.DataFrame(dict(hypothesis=hypotheses, reference=references))
data

,hypothesis,reference
0,맞아 그냥 모든 시험 채용이라 이런 데에서 공평하게 남자 몇 명 여자 몇 명 뽑겠다...,맞아 그냥 이렇게 시험 채용이나 이런 데에서 공평하게 남자 몇 명 여자 몇 명 뽑겠...
1,우리도 지금 보면 여자면명 꼭 뻗어야 된다고 이런 식으로 해서 뽑고는 있는데 또 막...,우리도 지금 보면은 어 여자 몇 명 꼭 뽑아야 된다고 이런 식으로 해서 뽑고는 있는...
2,남자는 예를 들어서 달리기를 100개를 해야 된다면 여자나 50개만 남아있게 가고 ...,남자는 예를 들어서 뭐 달리기를 뭐 100개를 해야 된다면은 여자는 한 50개만 하...
3,그래서 미국 같은 경우는 남자 여자 체력 기준을 똑같이 해가지고 똑같이 채용을 하거든.,"그래가지고 미국 같은 경우는 나, 남자 여자 체력 기준을 똑같이 해가지고 똑같이 채..."
4,그만큼 여자도 그만큼 능력이 되는 사람이 와서 일을 하니까 우리도 뭐 할 수 없는 ...,그만큼 여자도 인제 그만큼 능력이 되는 사람이 와서 일을 하니까 우리도 어 뭐라 할...
...,...,...
1995,주인이 잘못했어,음 음 주인이 잘못했어.\n
1996,그거는 동글병으로 갈 게 아니라 훈련사들 불러야 하는 거 아니에요.,그거는 동물 병원 갈 게 아니라 그 뭐야 뭐야 훈련사들 불러야 되는 거 아니야?\n
1997,야 내가 찾아... 그것도 찾아 봤거든? 한 번 부른데 30만원이죠?,야 그거 내가 찾아 l/ 그것도 찾아봤거든 한 번 부르는데 30만 원이 넘어. l/\n
1998,K-FK 훈련사처럼,c/ 그 개 개 훈련사처럼.\n
